In [1]:
# Uninstall old versions
!pip uninstall -y moviepy decorator

# Install latest MoviePy
!pip install moviepy


Found existing installation: decorator 5.2.1
Uninstalling decorator-5.2.1:
  Successfully uninstalled decorator-5.2.1
  Using cached moviepy-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached decorator-5.2.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached proglog-0.1.12-py3-none-any.whl.metadata (794 bytes)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached moviepy-2.2.1-py3-none-any.whl (129 kB)
Using cached decorator-5.2.1-py3-none-any.whl (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 18.3 MB/s  0:00:00 eta 0:00:01
Using cached proglog-0.1.12-py3-none-any.whl (6.3 kB)
Using cached imageio_ffmpeg-0.6.0-py3-none-macosx_11_0_arm64.whl (21.1 MB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
  Attempting uninstall: pillow
    Found existing installation: pillow 12.0.0
    Uninstalling pillow-12.0.0:
      Successfully uninstalled pillow-1

In [ ]:
import os
from pathlib import Path
from moviepy import VideoFileClip  # Direct import, no .editor
from datetime import datetime


def parse_timestamp(timestamp_str):
    """Convert MM:SS timestamp string to seconds."""
    try:
        parts = timestamp_str.split(':')
        if len(parts) != 2:
            raise ValueError("Timestamp must be in MM:SS format")

        minutes = int(parts[0])
        seconds = int(parts[1])

        if minutes < 0 or seconds < 0 or seconds >= 60:
            raise ValueError("Invalid time values")

        return minutes * 60 + seconds
    except Exception as e:
        raise ValueError(f"Invalid timestamp format '{timestamp_str}': {e}")


def extract_audio_from_video(video_path, start_time_str, end_time_str, output_filename=None):
    """
    Extract audio slice from video file.

    Args:
        video_path: Path to video file
        start_time_str: Start time in MM:SS format
        end_time_str: End time in MM:SS format
        output_filename: Optional custom output filename (without extension)

    Returns:
        Path to saved audio file
    """
    # Create output directory
    output_dir = Path("extracted_audio")
    output_dir.mkdir(exist_ok=True)

    # Parse timestamps
    start_seconds = parse_timestamp(start_time_str)
    end_seconds = parse_timestamp(end_time_str)

    # Generate output filename
    if output_filename:
        output_path = output_dir / f"{output_filename}.mp3"
    else:
        video_basename = Path(video_path).stem
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = output_dir / f"{video_basename}_slice_{timestamp}.mp3"

    print(f"\n{'='*70}")
    print(f"🎵 Video Audio Extractor")
    print(f"{'='*70}")
    print(f"📁 Video: {video_path}")
    print(f"⏰ Start: {start_time_str} ({start_seconds}s)")
    print(f"⏰ End: {end_time_str} ({end_seconds}s)")
    print(f"{'='*70}\n")

    # Load video and extract audio
    print(f"📂 Loading video...")
    video = VideoFileClip(video_path)

    print(f"🎬 Video duration: {video.duration:.2f} seconds")

    # Validate timestamps
    if start_seconds < 0 or end_seconds > video.duration:
        video.close()
        raise ValueError(f"Timestamps out of range. Video duration: {video.duration:.2f}s")

    if start_seconds >= end_seconds:
        video.close()
        raise ValueError("Start time must be before end time")

    # Extract audio slice
    print(f"✂️  Extracting audio from {start_seconds}s to {end_seconds}s...")
    audio_clip = video.subclipped(start_seconds, end_seconds).audio

    # Save audio
    print(f"💾 Saving audio to: {output_path}")
    audio_clip.write_audiofile(str(output_path), codec='mp3', bitrate='192k', logger=None)

    # Cleanup
    audio_clip.close()
    video.close()

    print(f"\n{'='*70}")
    print(f"✅ SUCCESS! Audio saved to: {output_path}")
    print(f"{'='*70}\n")

    return str(output_path)


# ============= USAGE EXAMPLE =============
if __name__ == "__main__":
    # Change these values for your video
    video_path = "your_video.mp4"
    start_time = "00:30"  # MM:SS format
    end_time = "01:00"

    # Optional: custom output name
    output_name = None  # or "my_audio_clip"

    try:
        audio_file = extract_audio_from_video(video_path, start_time, end_time, output_name)
        print(f"🎉 Done! Your audio is ready at: {audio_file}")
    except Exception as e:
        print(f"💥 Error: {e}")



🎵 Video Audio Extractor
📁 Video: your_video.mp4
⏰ Start: 00:30 (30s)
⏰ End: 01:00 (60s)

📂 Loading video...
💥 Error: 'your_video.mp4' not found


In [4]:
# Specify your local video file path
#video_path = "test_video_1_clean.mp4"  # or full path like "/content/video.mp4"
video_path = "data/raw/videos/test_video_1_clean.mp4"

tart_time = "00:30"
end_time = "01:00"

# Extract the audio slice
audio_file = extract_audio_from_video(video_path, start_time, end_time, output_name)


🎵 Video Audio Extractor
📁 Video: data/raw/videos/test_video_1_clean.mp4
⏰ Start: 00:30 (30s)
⏰ End: 01:00 (60s)

📂 Loading video...


FileNotFoundError: 'data/raw/videos/test_video_1_clean.mp4' not found